In [5]:
import scanpy as sc
import pandas as pd

In [3]:
adata = sc.read_h5ad('/Volumes/Castelo_Branco/christoffer/human_ms_xenium/data/trobisch/trobisch_raw_data.h5ad')

In [6]:
features = pd.read_csv('/Volumes/Castelo_Branco/christoffer/human_ms_xenium/data/trobisch/features.tsv',sep = '\t', header = None)

In [7]:
features = features.rename(columns = {0:'gene'}).set_index('gene')

In [8]:
adata.var = features

In [9]:
adata.write('../data/trobisch.h5ad')

In [13]:
human_genes = [
    "HIF1A",     # Hypoxia inducible factor 1 alpha
    "HK2",       # Hexokinase 2
    "LDHA",      # Lactate dehydrogenase A
    "LDHB",      # Lactate dehydrogenase B
    "SLC16A1",   # Monocarboxylate transporter 1 (MCT1)
    "SLC16A3",   # Monocarboxylate transporter 4 (MCT4)

        "PDK1",      # Pyruvate dehydrogenase kinase 1
    "PFKL",      # Phosphofructokinase, liver type
   
    "MFN1",      # Mitofusin 1
    "MFN2",      # Mitofusin 2
    "OPA1",      # OPA1 mitochondrial dynamin like GTPase

    "PKM",       # Pyruvate kinase M1/2
    "PPARGC1A",  # PGC-1α, transcriptional coactivator
    "SIRT2",     # Sirtuin 2

]

In [17]:
# 1) Normalize per cell (counts → CPM/TPM-like)
sc.pp.normalize_total(adata, target_sum=1e4)

# 2) Log-transform
sc.pp.log1p(adata)

In [32]:
list(adata.obs.Cluster.unique())

['OL-A',
 'L4-LRRK1-EN',
 'GRAN-EN',
 'motoEN',
 'WM-AS',
 'OL-C',
 'L2-L3-EN',
 'MYO5B-IN',
 'MG',
 'OPC',
 'OL-Stress',
 'L5-L6-SEMA3E-EN',
 'VSM',
 'GM-AS',
 'KIT-IN',
 'VIP-IN',
 'L5-L6-FRMD6-EN',
 'L3-EN',
 'T cells',
 'END',
 'SST-IN',
 'L5-L6-NTNG2-EN',
 'NPNT-IN',
 'L4-NPFFR2-EN',
 'L2-EN',
 'L4-GABRG1-EN',
 'STR',
 'PER',
 'CXCL14-IN',
 'L5-L6-HTR2C-EN',
 'L4-MME-EN',
 'Mix',
 'B cells',
 'BG',
 'PC',
 'WM-AS-GFPT2',
 'OL-B',
 'EPD']

In [31]:
adata[adata.obs.Cluster.str.contains('AS')].obs.Cluster.unique()

['WM-AS', 'GM-AS', 'WM-AS-GFPT2']
Categories (3, object): ['GM-AS', 'WM-AS', 'WM-AS-GFPT2']

In [42]:
import scanpy as sc
adata_sub = adata[adata.obs.Cluster.str.contains('OL')]
# Run differential expression between conditions
sc.tl.rank_genes_groups(
    adata_sub,
    groupby="condition",       # column in adata.obs
    reference="Control",       # baseline group
    method="wilcoxon",         # same as before (or "t-test", "logreg")
    use_raw=False              # set True if you want raw counts/log1p
)

# Extract results for "MS" vs "Control"
deg_condition = sc.get.rank_genes_groups_df(adata_sub, group="MS")
deg_condition.head()
print(deg_condition[deg_condition.names.isin(human_genes)].sort_values(by = 'logfoldchanges', ascending = False))

/Users/christoffer/miniconda3/envs/sc/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}


          names     scores  logfoldchanges         pvals     pvals_adj
354        LDHA   4.826467        2.902683  1.389763e-06  5.363621e-05
55          PKM  13.199093        2.162434  8.879772e-40  2.256135e-37
368        LDHB   4.712542        1.382749  2.446461e-06  9.056226e-05
335       SIRT2   5.042394        0.592548  4.597447e-07  1.889573e-05
5245   PPARGC1A   0.215882        0.436252  8.290799e-01  1.000000e+00
1289       OPA1   1.576155        0.241889  1.149902e-01  1.000000e+00
5247       MFN2   0.215720        0.224419  8.292061e-01  1.000000e+00
3203      HIF1A   0.527449        0.190919  5.978821e-01  1.000000e+00
6142       MFN1   0.146643        0.107079  8.834136e-01  1.000000e+00
31338      PFKL  -1.262108       -0.221035  2.069099e-01  1.000000e+00
29862      PDK1  -0.564284       -0.500240  5.725611e-01  1.000000e+00
28674   SLC16A3  -0.296447       -0.544179  7.668889e-01  1.000000e+00
31200   SLC16A1  -1.161782       -0.620345  2.453239e-01  1.000000e+00
28373 

In [43]:
import scanpy as sc
adata_sub = adata[adata.obs.Cluster.str.contains('AS')]
# Run differential expression between conditions
sc.tl.rank_genes_groups(
    adata_sub,
    groupby="condition",       # column in adata.obs
    reference="Control",       # baseline group
    method="wilcoxon",         # same as before (or "t-test", "logreg")
    use_raw=False              # set True if you want raw counts/log1p
)

# Extract results for "MS" vs "Control"
deg_condition = sc.get.rank_genes_groups_df(adata_sub, group="MS")
deg_condition.head()
print(deg_condition[deg_condition.names.isin(human_genes)].sort_values(by = 'logfoldchanges', ascending = False))

/Users/christoffer/miniconda3/envs/sc/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}


          names    scores  logfoldchanges         pvals     pvals_adj
516        LDHA  3.885164        1.543838  1.022607e-04  4.454050e-03
1662       MFN2  2.021855        1.179662  4.319130e-02  6.663063e-01
60          PKM  8.630652        0.971099  6.100304e-18  1.440789e-15
1228      SIRT2  2.445212        0.881832  1.447670e-02  2.910790e-01
1204       PFKL  2.476992        0.476750  1.324947e-02  2.714483e-01
6220    SLC16A1  0.665334        0.417432  5.058370e-01  1.000000e+00
2937       LDHB  1.361587        0.243989  1.733284e-01  1.000000e+00
1236      HIF1A  2.438872        0.222346  1.473319e-02  2.943420e-01
3495       OPA1  1.196517        0.122312  2.314950e-01  1.000000e+00
6724       PDK1  0.596398        0.107267  5.509096e-01  1.000000e+00
5350        HK2  0.792961        0.009611  4.278005e-01  1.000000e+00
13376   SLC16A3  0.126088       -0.009611  8.996621e-01  1.000000e+00
12395      MFN1  0.160924       -0.115226  8.721534e-01  1.000000e+00
33213  PPARGC1A -3.2

In [44]:
import scanpy as sc
adata_sub = adata[adata.obs.Cluster.str.contains('MG')]
# Run differential expression between conditions
sc.tl.rank_genes_groups(
    adata_sub,
    groupby="condition",       # column in adata.obs
    reference="Control",       # baseline group
    method="wilcoxon",         # same as before (or "t-test", "logreg")
    use_raw=False              # set True if you want raw counts/log1p
)

# Extract results for "MS" vs "Control"
deg_condition = sc.get.rank_genes_groups_df(adata_sub, group="MS")
deg_condition.head()
print(deg_condition[deg_condition.names.isin(human_genes)].sort_values(by = 'logfoldchanges', ascending = False))

/Users/christoffer/miniconda3/envs/sc/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}


          names    scores  logfoldchanges         pvals  pvals_adj
75          PKM  5.643096        1.594943  1.670194e-08   0.000005
325        LDHA  2.915569        1.587120  3.550406e-03   0.276273
862        LDHB  1.672711        1.451731  9.438426e-02   1.000000
2960      SIRT2  0.741034        0.728998  4.586728e-01   1.000000
3282       MFN2  0.677084        0.715024  4.983529e-01   1.000000
1132       OPA1  1.429399        0.363140  1.528897e-01   1.000000
10476  PPARGC1A  0.104167        0.289378  9.170371e-01   1.000000
2620    SLC16A3  0.820771        0.201383  4.117769e-01   1.000000
9042       MFN1  0.162257        0.017590  8.711036e-01   1.000000
10254      PFKL  0.117169       -0.069275  9.067259e-01   1.000000
29306   SLC16A1 -0.029411       -0.113418  9.765365e-01   1.000000
33106     HIF1A -0.860950       -0.213620  3.892654e-01   1.000000
32590      PDK1 -0.322719       -0.374270  7.469081e-01   1.000000
33026       HK2 -0.687009       -0.550349  4.920768e-01   1.00